Two Sigma Housing Interest

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

EDA

In [ ]:
train = pd.read_json("../input/train.json")

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
#find correlations

In [ ]:
sns.pairplot(data=train,hue='interest_level')

In [ ]:
sns.stripplot(x='interest_level',y='listing_id',data=train,jitter=True)

In [ ]:
sns.lmplot(x='bathrooms',y='bedrooms',data=train,hue='interest_level')

In [ ]:
#remove geographical outliers
train = train[train['latitude']>35]
sns.lmplot(x='latitude',y='longitude',data=train,hue='interest_level',fit_reg=False)

In [ ]:
sns.lmplot(x='latitude',y='longitude',data=train[(train['latitude']<41.5) & (train['latitude']>40.2)],hue='interest_level',fit_reg=False,scatter_kws={'alpha':0.2})

In [ ]:
sns.lmplot(x='latitude',y='longitude',data=train[train['interest_level']=='medium'],fit_reg=False)

In [ ]:
#remove bathroom outliers
train = train[train['bathrooms']<10]
sns.stripplot(x='interest_level',y='bathrooms',data=train,jitter=True)

In [ ]:
sns.stripplot(x='interest_level',y='bedrooms',data=train,jitter=True)

In [ ]:
#remove price outliers
train = train[train['price']<1000000]
sns.stripplot(x='interest_level',y='price',data=train,jitter=True)

In [ ]:
sns.barplot(x='interest_level',y='price',data=train)

In [ ]:
sns.barplot(x='interest_level',y='bedrooms',data=train)

In [ ]:
sns.barplot(x='interest_level',y='bathrooms',data=train)

In [ ]:
train['latlong']=train['latitude']*train['longitude']
sns.boxplot(x='interest_level',y='longitude',data=train)

In [ ]:
train.columns

In [ ]:
from datetime import datetime
dates = train['created'].apply(lambda x: x[0:10])

In [ ]:
dates = dates.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
dates

In [ ]:
train['dates']=dates

In [ ]:
sns.countplot(train['dates'],orient='v')

In [ ]:
train.columns

In [ ]:
train['features'].head()

In [ ]:
train['feature_length']=train['features'].apply(len)

In [ ]:
sns.boxplot(x='interest_level',y='feature_length',data=train)

In [ ]:
# extract all features
feature_vector = []
for feature_list in train['features']:
    for feature in feature_list:
           feature_vector.append(feature)

In [ ]:
len(feature_vector)

In [ ]:
#extract all feature words
word_counts = []
for i in feature_vector:
    for j in i.split(' '):
        word_counts.append(j)

In [ ]:
len(word_counts)

In [ ]:
word_series = pd.Series(word_counts)

In [ ]:
word_series.value_counts()

In [ ]:
# check what features are most valuable
# flatten lists in word list
def flatten(S):
    wordlist = []
    for i in S:
        if len(i.split(' ')) > 1:
            for j in i.split(' '):
                wordlist.append(j)
        else:
            wordlist.append(i)
    return wordlist

In [ ]:
# test flatten
l = train['features'].iloc[1]
print(flatten(l))

In [ ]:
train['flattened'] = train['features'].apply(flatten)

In [ ]:
train['allowed'] = train['flattened'].apply(lambda x: 'Allowed' in x)

In [ ]:
sum(train['allowed'])

In [ ]:
sns.countplot(x='allowed',data=train,hue='interest_level')

In [ ]:
def word_significance(word):
    train[word] = train['flattened'].apply(lambda x: word in x)
    sns.countplot(x=word,data=train,hue='interest_level')

In [ ]:
def flatten(S):
    wordlist = []
    for i in S:
        if len(i.split(' ')) > 1:
            for j in i:
                wordlist.append(j)
        else:
            wordlist.append(i)
    return wordlist